In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 8
height = 8
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 60

puzzle.naming = {
    None: '_',
}

puzzle.domain_program += """
    num(0..9).
    value(N) :- num(N).
    guessed_number(square_sum_min,12).
    guessed_number(square_sum_max,12).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    %:- value(V), 4 { solution(C,V) : solution(C,V) }.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""

puzzle.solution_program += """
    square_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height,
        Col < board_width.
    square_rel(c(Row,Col),c(Row,Col)) :- square_cell(c(Row,Col)).
    square_rel(c(Row,Col),c(Row,Col+1)) :- square_cell(c(Row,Col)).
    square_rel(c(Row,Col),c(Row+1,Col)) :- square_cell(c(Row,Col)).
    square_rel(c(Row,Col),c(Row+1,Col+1)) :- square_cell(c(Row,Col)).
    square_check(C1) :- square_cell(C1),
        guessed_number(square_sum_min,Smin),
        guessed_number(square_sum_max,Smax),
        S = #sum { V2,C2 : solution(C2,V2), square_rel(C1,C2) },
        Smin <= S, S <= Smax.
    :- square_cell(C), not square_check(C).
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

..................................................
Stopped after solving timeout..
Solving time: 61.02 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|_|_|_|_|_|_|_|_|
|_|_|_|_|_|_|_|2|
|_|_|_|_|_|_|_|_|
|_|4|8|1|_|1|_|3|
|6|_|_|_|7|_|7|_|
|_|_|_|_|2|_|_|_|
|6|_|_|_|_|_|_|_|
|_|_|_|0|_|0|_|_|

square_sum_min = 12
square_sum_max = 12


In [5]:
print(puzzle.pretty_repr_solution())

|6|0|0|3|7|3|7|1|
|3|3|9|0|2|0|2|2|
|6|0|0|3|7|3|7|1|
|2|4|8|1|1|1|1|3|
|6|0|0|3|7|3|7|1|
|3|3|9|0|2|0|2|2|
|6|0|0|3|7|3|7|1|
|3|3|9|0|2|0|2|2|
